# 임포트 부분

In [29]:
import pandas as pd
from konlpy.tag import Okt
from tqdm import trange, notebook
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
import re
from sklearn.feature_extraction.text import CountVectorizer

# 데이터 가져오기

In [2]:
a = pd.read_excel('total_최종.xlsx')
a

,data,label
0,Game Designer(게임 디자이너)\n\n비디오 게임 제작자의 부류. 영어로는...,game
1,"게임크리에이터, 게임디자이너\n게임기획자는 PC 게임, 네트워크 게임 등 게임용 소...",game
2,"게임기획자란?\n실제 게임제작, 게임프로그래밍이나 그래픽 작업에 들어가기 전 어떤 ...",game
3,"하는일\n컴퓨터 게임의 아이템과 스토리, 캐릭터 등을 구성 및 기획하고 제작 과정을...",game
4,게임기획자\n\n게임 기획자는 게임 회사의 기획팀에 속해서 일하는 사람들을 말합니다...,game
...,...,...
118,안녕하세요. 자기소개 부탁드립니다.\n안녕하세요! 개발자 고영정이라고 합니다. 토스...,mobile
119,버즈빌에 대한 간단한 소개도 부탁드립니다.\n버즈빌은 글로벌 잠금 화면 광고 플랫폼...,mobile
120,반갑습니다. 자기소개 부탁 드립니다.\n안녕하세요. 캐시슬라이드에서 제품길드의 클라...,mobile
121,안녕하세요. 간단한 자기소개 부탁 드립니다.\n안녕하세요! 와이즈솔루션 테크니컬 파...,mobile


# 특수문자 제거

In [3]:
a['data'] = a['data'].str.replace("[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9 ]","")
a

<ipython-input-3-f18787e8bf6f>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  a['data'] = a['data'].str.replace("[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9 ]","")


,data,label
0,Game Designer게임 디자이너비디오 게임 제작자의 부류 영어로는 게임 디자이...,game
1,게임크리에이터 게임디자이너게임기획자는 PC 게임 네트워크 게임 등 게임용 소프트웨어...,game
2,게임기획자란실제 게임제작 게임프로그래밍이나 그래픽 작업에 들어가기 전 어떤 게임을 ...,game
3,하는일컴퓨터 게임의 아이템과 스토리 캐릭터 등을 구성 및 기획하고 제작 과정을 관리...,game
4,게임기획자게임 기획자는 게임 회사의 기획팀에 속해서 일하는 사람들을 말합니다 보통 ...,game
...,...,...
118,안녕하세요 자기소개 부탁드립니다안녕하세요 개발자 고영정이라고 합니다 토스랩은 잔디라...,mobile
119,버즈빌에 대한 간단한 소개도 부탁드립니다버즈빌은 글로벌 잠금 화면 광고 플랫폼을 만...,mobile
120,반갑습니다 자기소개 부탁 드립니다안녕하세요 캐시슬라이드에서 제품길드의 클라이언트 클...,mobile
121,안녕하세요 간단한 자기소개 부탁 드립니다안녕하세요 와이즈솔루션 테크니컬 파트에서 일...,mobile


# 형태소 분석, 불용어 제거

In [98]:
stopwords = ['을', '를', '이', '가', '은', '는','이','그','있','n','nn','t','의','되','라','한','다','하','더','에','서','나',
            '과','고','며','와','지','해','도','지','못','안','않','로','들','된','것','에는','안녕',
            '반갑','내','등','할','자','있다','','직','군','위','식','각','출','창','시','수','및','신','영','화','살','여','양','송','대','데',
            '연','구','살','장','또','적','D','력','인','향','년','후','생','기','봐','문','니','사','본','제','작','관','일','분','야',
            '중','거','명','어','개','타','제','또는','요','전','몇','중','에서','있는데요','된다','또한','되','돼','않','으로','만','있습니다','하는','하는것이',
            '만','푹','지금','하는','곳','로는','한다','이다','에서는','이라','고하','에서보다','이라고도','경','제등','이서','이라는',
            '가는','되는','마다','야하며','야한다','필요하다','되었을','필요한다','게','하고','좋아하고','원하는','통해','하여이에','고도',
            '세','드리지','않습니다','잉','님','합니다']

okt = Okt()
tokenized_data = []

for sentence in notebook.tqdm(a['data']):  # 프로그레스 바
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    #temp_X = [word for word in temp_X if len(word) > 1] # 한글자는 제거
    tokenized_data.append(temp_X)

  0%|          | 0/123 [00:00<?, ?it/s]

# array 변환

In [99]:
x = np.array(tokenized_data)
y = a.label.values

<ipython-input-99-609f2ddec93a>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x = np.array(tokenized_data)


# y라벨 변환

In [100]:
le = LabelEncoder()
y = le.fit_transform(y)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]


# 트레인 테스트 나누기

In [128]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y)

# 토큰화

In [129]:
# 토큰화max_len = 500
tok = Tokenizer()
tok.fit_on_texts(x_train)

# x_train text_to_sequences, pad_sequence 작업

In [130]:
sequences1 = tok.texts_to_sequences(x_train)
sequences2 = tok.texts_to_sequences(x_test)
sequences_matrix1 = sequence.pad_sequences(sequences1, maxlen=max_len)
sequences_matrix2 = sequence.pad_sequences(sequences2, maxlen=max_len)

# 나이브베이즈 모델

In [131]:
NB = MultinomialNB()
NB.fit(sequences_matrix1, y_train)
pred = NB.predict(sequences_matrix2)
acc = accuracy_score(pred, y_test)
print(acc)

0.44


# tf-idf를 이용한 정확도 향상

In [135]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
x_train_tf = tfidf.fit_transform(sequences_matrix1)
x_test_tf = tfidf.fit_transform(sequences_matrix2)

NB.fit(x_train_tf, y_train)
pred = NB.predict(x_test_tf)
acc = accuracy_score(pred, y_test)
print(acc)

0.4
